# Data preprocessing



In [1]:
import nltk
nltk.download('stopwords')
import re
import string
from nltk.corpus import stopwords

# Load Bengali stopwords
stop_words = set(stopwords.words('bengali'))

def preprocess(text):
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove special characters
    text = re.sub(r"[^\u0980-\u09FF\s]+", "", text)

    # Tokenize text into list of words
    words = nltk.word_tokenize(text)

    # Remove stop words and non-Bengali words
    words = [w for w in words if w not in stop_words and re.match(r'^[\u0980-\u09FF]+$', w)]

    # Join words into a cleaned sentence
    cleaned_text = ' '.join(words)

    return cleaned_text


# Model trained


In [12]:

from gensim.models import Word2Vec
import json
import os

# Define the paths to the JSON files
json_paths= ['D:\project495\data\itte-part-0.json', 'D:\project495\data\itte-part-1.json','D:\project495\data\itte-part-2.json','D:\project495\data\itte-part-3.json','D:\project495\data\itte-part-4.json','D:\project495\data\jug-part-0.json','D:\project495\data\jug-part-1.json','D:\project495\data\jug-part-2.json','D:\project495\data\jug-part-3.json','D:\project495\data\pro-part-0.json','D:\project495\data\pro-part-1.json','D:\project495\data\pro-part-2.json']

# Preprocess text and concatenate sentences
sentences = []
for json_path in json_paths:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        for item in data:
            gen_type = item.get('gen_type', '')
            headline = item.get('headline', '')
            additional_info = item.get('additional_info', '')
            body = item.get('body', '')
            tags = ' '.join(item.get('tags', []))

            text = f'{gen_type} {headline} {additional_info} {body} {tags}'
            cleaned_text = preprocess(text)
            words = cleaned_text.split()

            sentences.append(words)

# Build vocabulary and train Word2Vec model
model = Word2Vec(sentences, vector_size=300, min_count=1, workers=4)
model.train(sentences, total_examples=len(sentences), epochs=10)

# Save model
model.save('model.bin')


# Model evaluation


In [2]:

import gensim
import os
import json
import re
import string
import numpy as np
from gensim.models import Word2Vec
model = KeyedVectors.load('model.bin', mmap='r')
google_news = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# Bengali word to English word neighbor cosine similarity



In [ ]:
 
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


# Define a list of 500 Bengali words and English words
bengali_words = ['অজ্ঞতা', 'অনুমতি', 'অনুরোধ', 'অন্ধকার', 'অন্যায়', 'অশিক্ষা', 'আকাশ', 'আগুন', 'আলো', 'কফি', 'ক্ষতি', 'খরা', 'গরম', 'গাছ', 'ঘর', 'ঘৃণা', 'চা', 'জিজ্ঞাসা', 'জ্ঞান', 'ঝঞ্ঝা', 'ঝড়', 'ঝুঁকি', 'দারিদ্র্য', 'দুঃখ', 'দুধ', 'দেশ', 'নদী', 'নির্দেশ', 'নিশ্চয়তা', 'নিষেধাজ্ঞা', 'ন্যায়', 'পানি', 'পাহাড়', 'প্রেম', 'ফল', 'ফুল', 'বন্যা', 'বাতাস', 'বাধা', 'বিশ্বাস', 'বৃষ্টি', 'ব্যয়', 'ভাত', 'ভূমিকম্প', 'মাছ', 'মানুষ', 'মাংস', 'মিথ্যা', 'রুটি', 'লাভ', 'শিক্ষা', 'শীত', 'সঞ্চয়', 'সত্য', 'সন্দেহ', 'সমস্যা', 'সমাধান', 'সমুদ্র', 'সম্পদ', 'সাহায্য', 'সুখ', 'সুযোগ', 'আসতে', 'কথা', 'ঘুমানো', 'চলতে', 'দৌড়াতে', 'খেলতে', 'পড়া', 'দূরে', 'ওপর', 'নীচে', 'মেজাজ', 'চলা', 'আলো', 'খেলা', 'পথ', 'বদল', 'ভোর', 'সমৃদ্ধি', 'বল', 'যুদ্ধ', 'প্রশ্ন', 'দরজা', 'খোদা', 'মন', 'খাদ্য', 'বসা', 'রং', 'মুখ', 'পর্ব', 'পাথর', 'সৌন্দর্য', 'সময়', 'পথিক', 'শাখা', 'মাস', 'খোঁজা', 'বাঁধা', 'সুন্দর', 'লম্বা', 'ঝগড়া', 'শান্তি', 'ভালোবাসা', 'আনন্দ', 'বন্ধু', 'শত্রু', 'শিক্ষক', 'ছাত্র', 'বাড়ি', 'বাইরে', 'বাজার', 'দোকান', 'ব্যবসা', 'চাকরি', 'সাহস', 'ভয়', 'বুদ্ধি', 'রাজ্য', 'শহর', 'গ্রাম', 'উপত্যকা', 'রাত', 'বীজ', 'পাতা', 'প্রাণী', 'কক্ষ', 'ভাই', 'বোন', 'রাজা', 'রাণী', 'মেয়ে', 'ছেলে', 'জানাান', 'আসা', 'শোনা', 'লুকানো', 'বোনা', 'হারানো', 'গলানো', 'দৌড়ানো', 'পাঠানো', 'দাঁড়ানো', 'পরিধান', 'ঝাঁকানো', 'বাজানো', 'চালানো', 'শেখা', 'কাটা', 'পাওয়া', 'খাওয়ানো', 'খাওয়া', 'টানা', 'ভর্ৎসনা', 'পোড়ানো', 'কেনা', 'পৌছানো', 'মারা', 'ভাঙা', 'আনা', 'ডাকা', 'প্রকাশ', 'বছর', 'বিরুদ্ধে', 'নৈতিক', 'ভাগ্যবান', 'বন্দী', 'ওড়া', 'লাফানো', 'হাসা', 'সক্ষমতা', 'অনুপস্থিতি', 'প্রাচুর্য', 'প্রবেশ', 'দুর্ঘটনা', 'হিসাব', 'ব্যথা', 'ভর্তি', 'সুবিধা', 'উপদেশ', 'স্নেহ', 'চুক্তি', 'ক্রোধ', 'দুশ্চিন্তা', 'অনুমান', 'প্রতিশ্রুতি', 'সচেতনতা', 'অবিবাহিত', 'মালপত্র', 'দেউলে', 'প্রতিবন্ধকতা', 'ভিত্তি', 'অস্তিত্ব', 'আশীর্বাদ', 'উপকার', 'বিশ্বাসঘাতক', 'তিক্ততা', 'অবরোধ', 'সাহসিকতা', 'বিদ্যুৎ', 'অনুষ্ঠিত', 'অংশ', 'সংবাদ', 'আটক', 'উপস্থিত', 'একটা', 'নিশ্চিত', 'ছাড়া', 'হত্যা', 'পাশাপাশি', 'রাজনৈতিক', 'নির্বাহী', 'অনলাইন', 'অবস্থায়', 'প্রার্থী', 'নারী', 'প্রতিনিধি', 'গ্রহণ', 'দক্ষিণ', 'সম্ভব', 'আদালত', 'নেয়া', 'উন্নয়ন', 'কেন্দ্রীয়', 'ঘটনার', 'প্রেসিডেন্ট', 'কমিটি', 'সৃষ্টি', 'কম', 'অর্থ', 'পরীক্ষা', 'অতিরিক্ত', 'অভিযান', 'সংখ্যা', 'পালন', 'সড়ক', 'সংসদ', 'মার্কিন', 'হামলা', 'আইন', 'বর্তমান', 'স্ত্রী', 'দুপুরে', 'বিক্রি', 'ভোট', 'বিশ্ব', 'সহকারী', 'আয়োজন', 'মেডিকেল', 'শিশু', 'সম্পর্কে', 'উল্লেখ', 'দাম', 'আবেদন', 'সূত্র', 'অবস্থান', 'সচিব', 'প্রমুখ', 'পক্ষ', 'সংশ্লিষ্ট', 'দ্রুত', 'বিপক্ষে', 'নিরাপত্তা', 'প্রতিষ্ঠান', 'ছবি', 'পদে', 'কার্যক্রম', 'যোগাযোগ', 'ছোট', 'নিজ', 'নেন', 'এলাকা', 'আহ্বান', 'বাজারে', 'আগের', 'আশা', 'অতিথি', 'এসেছে', 'দশমিক', 'সামাজিক', 'বিচার', 'সারা', 'মূল', 'কেন্দ্র', 'নির্মাণ', 'কর্মসূচি', 'যোগ', 'শিকার', 'আসা', 'বিভাগ', 'ছয়', 'জয়', 'নিয়োগ', 'তিনটি', 'লোকজন', 'গঠন', 'চলতি', 'পরিমাণ', 'পরিবার', 'বেলা', 'জড়িত', 'সংস্থা', 'উইকেট', 'করেছি', 'হামলার', 'মালিক', 'ছেড়ে', 'মন্ত্রী', 'স্কুল', 'বাহিনীর', 'অফিস', 'মহানগর', 'ব্যক্তি', 'বাড়ির', 'নির্বাচিত', 'হাত', 'নিয়েছে', 'মধ্য', 'আসছে', 'পরিষদ', 'নেয়', 'অফিসার', 'ভেঙে', 'মৃত', 'ধারণা', 'প্রকল্প', 'প্রতিটি', 'গাড়ি', 'বাস', 'জীবন', 'উদ্যোগ', 'বেসরকারি', 'কমিশন', 'সিনিয়র', 'পৌর', 'হারিয়ে', 'ব্যাপক', 'হার', 'পরিবেশ', 'অধিনায়ক', 'ভাগ', 'ফেলে', 'করায়', 'বৃদ্ধি', 'দায়ের', 'জমা', 'উচ্চ', 'প্রস্তাব', 'বেড়ে', 'জমি', 'শহীদ', 'সভা', 'গান', 'লক্ষ্য', 'কৃষি', 'মহিলা', 'অবৈধ', 'পুরো', 'উদ্বোধন', 'চলাচল', 'মন্তব্য', 'বিক্ষোভ', 'বৈঠক', 'ঘণ্টা', 'দীর্ঘ', 'খাবার', 'ফুটবল', 'সেরা', 'অব্যাহত', 'অ্যাডভোকেট', 'সরাসরি', 'প্রয়োজনীয়', 'অধিকার', 'গোলাম', 'গোল', 'সংগঠন', 'কমে', 'বিমান', 'স্বামী', 'বাহিনী', 'বেড়েছে', 'নিয়মিত', 'লেখা', 'বিরোধী', 'চূড়ান্ত', 'মুক্তি', 'ক্ষমতা', 'স্বীকার', 'পরিচয়', 'একাধিক', 'সঠিক', 'পারেনি', 'নেতারা', 'গভীর', 'দিবস', 'রক্ষা', 'জারি', 'আদেশ', 'ভিডিও', 'কার্যকর', 'প্রথমে', 'কেটে', 'ভাড়া', 'স্থায়ী', 'কিলোমিটার', 'সম্পন্ন', 'পরামর্শ', 'কঠিন', 'ভাইস', 'পথ', 'ভুল', 'অর্জন', 'শীর্ষ', 'গোয়েন্দা', 'অপর', 'অনুমোদন', 'সর্বোচ্চ', 'শতাধিক', 'প্রেস', 'সংকট', 'ক্ষতিগ্রস্ত', 'দখল', 'নির্যাতন', 'সর্বশেষ', 'স্বাধীনতা', 'এবারের', 'গ্যাস', 'প্রতিবাদ', 'প্রমাণ', 'বোর্ড', 'মূল্য', 'চাপ', 'বয়স', 'বিকাল', 'কঠোর', 'সমর্থন', 'বিয়ে', 'স্বপ্ন', 'সম্ভাবনা', 'সমাবেশ', 'চন্দ্র', 'অপরাধ', 'জরিমানা', 'জন্ম', 'দৈনিক', 'ক্লাব', 'ব্যক্তিগত', 'ওষুধ', 'নির্ধারিত', 'সেনা', 'হাজির', 'শাখার', 'বিজ্ঞান', 'ভেতরে', 'কোম্পানি', 'আয়', 'জয়ের', 'চিঠি', 'নির্ধারণ', 'পার', 'বয়সী', 'শিশুদের', 'গুরুত্ব', 'উল্লেখ্য', 'শুভ', 'চরম', 'বিনিয়োগ', 'সন্তান', 'আপিল', 'বাড়ছে', 'জনপ্রিয়', 'উন্নত', 'স্থগিত', 'পাস', 'রাস্তা', 'পদ', 'দূর', 'মিনিট', 'স্থাপন', 'ব্যর্থ', 'নিজস্ব', 'শক্তিশালী', 'ভূমি', 'আশঙ্কা', 'জোর', 'শেয়ার', 'একসঙ্গে', 'প্রবাসী', 'মত', 'মাটি', 'বারবার', 'প্রাণ', 'পরিচালিত', 'কম্পিউটার', 'দমন', 'সার্বিক', 'নিয়েই', 'সক্ষম', 'সভাপতিত্ব', 'পরের', 'ওয়ার্ড', 'সমাজ', 'রাজি', 'সংযোগ', 'পররাষ্ট্র']  # Add 500 Bengali words here
english_words = ['ignorance', 'permission', 'request', 'darkness', 'injustice', 'illiteracy', 'sky', 'fire', 'light', 'coffee', 'damage', 'drought', 'hot', 'tree', 'house', 'hatred', 'tea', 'ask', 'knowledge', 'tempest', 'storm', 'risk', 'poverty', 'sadness', 'milk', 'country', 'river', 'instruction', 'certainty', 'prohibition', 'justice', 'water', 'mountain', 'love', 'fruit', 'flower', 'flood', 'air', 'hindrance', 'faith', 'rain', 'expense', 'rice', 'earthquake', 'fish', 'human', 'meat', 'falsehood', 'bread', 'gain', 'education', 'cold', 'saving', 'truth', 'doubt', 'problem', 'solution', 'sea', 'wealth', 'help', 'happiness', 'opportunity', 'come', 'speak', 'sleep', 'walk', 'run', 'play', 'read', 'far', 'above', 'below', 'mood', 'walk', 'light', 'game', 'road', 'change', 'morning', 'prosperity', 'strength', 'war', 'question', 'door', 'dig', 'mind', 'food', 'sit', 'color', 'face', 'episode', 'stone', 'beauty', 'time', 'traveler', 'branch', 'month', 'search', 'tie', 'beautiful', 'tall', 'quarrel', 'peace', 'love', 'joy', 'friend', 'foe', 'teacher', 'student', 'home', 'outside', 'market', 'shop', 'business', 'job', 'courage', 'fear', 'intelligence', 'state', 'city', 'village', 'valley', 'night', 'seed', 'leaf', 'animal', 'room', 'brother', 'sister', 'king', 'queen', 'girl', 'boy', 'inform', 'come', 'hear', 'hide', 'knit', 'lose', 'melt', 'run', 'send', 'stand', 'wear', 'shake', 'ring', 'lead', 'learn', 'hew', 'get', 'feed', 'eat', 'draw', 'chide', 'burn', 'buy', 'arrive', 'beat', 'break', 'bring', 'call', 'expression', 'year', 'against', 'ethical', 'lucky', 'captive', 'fly', 'jump', 'laugh', 'ability', 'absence', 'abundance', 'access', 'accident', 'account', 'ache', 'admission', 'advantage', 'advice', 'affection', 'agreement', 'anger', 'anxiety', 'assume', 'assurance', 'awareness', 'bachelor', 'baggage', 'bankrupt', 'barrier', 'basement', 'being', 'benediction', 'benefit', 'betrayer', 'bitterness', 'blockade', 'boldness', 'electricity', 'held', 'part', 'news', 'detention', 'present', 'one', 'sure', 'except', 'murder', 'alongside', 'political', 'executive', 'online', 'condition', 'candidate', 'woman', 'representative', 'accept', 'south', 'possible', 'court', 'take', 'development', 'central', 'event', 'president', 'committee', 'creation', 'less', 'money', 'test', 'extra', 'campaign', 'number', 'keeping', 'road', 'parliament', 'us', 'attack', 'law', 'current', 'wife', 'noon', 'sale', 'vote', 'world', 'assistant', 'arrangement', 'medical', 'child', 'about', 'mention', 'price', 'application', 'formula', 'location', 'secretary', 'others', 'side', 'concerned', 'quickly', 'against', 'safety', 'institution', 'picture', 'position', 'activities', 'communication', 'small', 'own', 'take', 'area', 'invitation', 'market', 'previous', 'hope', 'guest', 'come', 'decimal', 'social', 'trial', 'all', 'principal', 'center', 'construction', 'schedule', 'addition', 'victim', 'coming', 'division', 'six', 'victory', 'appointment', 'three', 'people', 'formation', 'current', 'quantity', 'family', 'afternoon', 'involved', 'organization', 'wicket', 'done', 'attack', 'owner', 'left', 'minister', 'school', 'army', 'office', 'metropolitan', 'person', 'house', 'elected', 'hand', 'taken', 'middle', 'coming', 'council', 'takes', 'officer', 'broken', 'dead', 'idea', 'project', 'every', 'car', 'bus', 'life', 'initiative', 'private', 'commission', 'senior', 'municipal', 'lost', 'widespread', 'loss', 'environment', 'captain', 'share', 'throwing', 'causes', 'growth', 'filed', 'deposited', 'high', 'proposal', 'increased', 'land', 'martyr', 'meeting', 'song', 'goal', 'agriculture', 'woman', 'illegal', 'whole', 'opening', 'movement', 'comment', 'protest', 'meeting', 'hour', 'long', 'food', 'football', 'best', 'continuous', 'advocate', 'directly', 'necessary', 'right', 'slave', 'goal', 'organization', 'decreased', 'airplane', 'husband', 'forces', 'increased', 'regular', 'written', 'opponent', 'final', 'freedom', 'power', 'admitted', 'identity', 'multiple', 'correct', 'couldnt', 'leaders', 'deep', 'day', 'protection', 'issued', 'order', 'video', 'effective', 'first', 'cut', 'rent', 'permanent', 'kilometer', 'complete', 'advice', 'hard', 'vice', 'way', 'wrong', 'achievement', 'top', 'detective', 'other', 'approval', 'highest', 'hundreds', 'press', 'crisis', 'injured', 'occupation', 'torture', 'latest', 'independence', 'this', 'gas', 'protest', 'proof', 'board', 'value', 'pressure', 'age', 'afternoon', 'harsh', 'support', 'wedding', 'dream', 'possibility', 'assembly', 'moon', 'crime', 'fine', 'birth', 'daily', 'club', 'private', 'medicine', 'scheduled', 'army', 'present', 'branch', 'science', 'inside', 'company', 'income', 'victory', 'letter', 'determination', 'can', 'aged', 'children', 'importance', 'noteworthy', 'auspicious', 'extreme', 'investment', 'child', 'appeal', 'increasing', 'popular', 'developed', 'adjourned', 'pass', 'street', 'post', 'far', 'minute', 'establishment', 'failure', 'own', 'strong', 'land', 'fear', 'force', 'share', 'together', 'expat', 'opinion', 'soil', 'repeatedly', 'life', 'managed', 'computer', 'suppression', 'overall', 'with', 'able', 'presidency', 'next', 'ward', 'society', 'agreed', 'connection', 'foreign']  # Add 500 English words here

# Lists to store the results
bengali_word_list = []
bengali_neighbor_list = []
english_neighbor_list = []
cosine_similarity_list = []
positive_cosine_similarity_list = []
negative_cosine_similarity_list = []
# Calculate and store the results for all 500 words
for bengali_word, english_word in zip(bengali_words, english_words):
    # Find the top 1 nearest neighbor of the Bengali word in the Bengali Word2Vec model
    bengali_neighbors = model.wv.most_similar([bengali_word], topn=2)
    bengali_vector = model.wv.get_vector(bengali_word)  # Extract the Bengali word's vector
    
    # Get the English word's vector
    english_vector = google_news.get_vector(english_word)
    
    # Find the top 10 nearest neighbors of the English word in the English Word2Vec model
    english_neighbors = google_news.most_similar([english_vector], topn=10)
    
    # Get the English word's neighbors' vectors
    english_neighbors_vectors = [google_news.get_vector(word) for word, _ in english_neighbors]
    
    # Calculate the cosine similarities between the Bengali word's vector and the English word's neighbors' vectors
    cosine_similarities = cosine_similarity(np.array([bengali_vector]), np.array(english_neighbors_vectors))

    for eng_neighbor, sim in zip([word for word, _ in english_neighbors], cosine_similarities[0]):
        bengali_word_list.append(bengali_word)
        bengali_neighbor_list.append(bengali_neighbors[0][0])
        english_neighbor_list.append(eng_neighbor)
        cosine_similarity_list.append(sim)
        if sim >= 0:
            positive_cosine_similarity_list.append(sim)
            negative_cosine_similarity_list.append(np.nan)
        else:
            positive_cosine_similarity_list.append(np.nan)
            negative_cosine_similarity_list.append(sim)

# Create DataFrames for positive and negative cosine similarities
positive_results = {
    'Bengali Word': bengali_word_list,
    'Bengali Neighbor': bengali_neighbor_list,
    'English Neighbor': english_neighbor_list,
    'Cosine Similarity': positive_cosine_similarity_list
}
negative_results = {
    'Bengali Word': bengali_word_list,
    'Bengali Neighbor': bengali_neighbor_list,
    'English Neighbor': english_neighbor_list,
    'Cosine Similarity': negative_cosine_similarity_list
}
positive_df = pd.DataFrame(positive_results)
negative_df = pd.DataFrame(negative_results)

# Create an Excel writer
writer = pd.ExcelWriter('bangla_cosine_similarities_01.xlsx', engine='xlsxwriter')

# Save DataFrames to separate sheets in the Excel file
positive_df.to_excel(writer, sheet_name='Positive_Cosine_Similarity', index=False)
negative_df.to_excel(writer, sheet_name='Negative_Cosine_Similarity', index=False)

# Close the writer to save the changes
writer.close()



# Neighbor cosine similarity (English neighbor to Bengali neighbor)



In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Define a list of 500 Bengali words and English words
bengali_words = ['অজ্ঞতা', 'অনুমতি', 'অনুরোধ', 'অন্ধকার', 'অন্যায়', 'অশিক্ষা', 'আকাশ', 'আগুন', 'আলো', 'কফি', 'ক্ষতি', 'খরা', 'গরম', 'গাছ', 'ঘর', 'ঘৃণা', 'চা', 'জিজ্ঞাসা', 'জ্ঞান', 'ঝঞ্ঝা', 'ঝড়', 'ঝুঁকি', 'দারিদ্র্য', 'দুঃখ', 'দুধ', 'দেশ', 'নদী', 'নির্দেশ', 'নিশ্চয়তা', 'নিষেধাজ্ঞা', 'ন্যায়', 'পানি', 'পাহাড়', 'প্রেম', 'ফল', 'ফুল', 'বন্যা', 'বাতাস', 'বাধা', 'বিশ্বাস', 'বৃষ্টি', 'ব্যয়', 'ভাত', 'ভূমিকম্প', 'মাছ', 'মানুষ', 'মাংস', 'মিথ্যা', 'রুটি', 'লাভ', 'শিক্ষা', 'শীত', 'সঞ্চয়', 'সত্য', 'সন্দেহ', 'সমস্যা', 'সমাধান', 'সমুদ্র', 'সম্পদ', 'সাহায্য', 'সুখ', 'সুযোগ', 'আসতে', 'কথা', 'ঘুমানো', 'চলতে', 'দৌড়াতে', 'খেলতে', 'পড়া', 'দূরে', 'ওপর', 'নীচে', 'মেজাজ', 'চলা', 'আলো', 'খেলা', 'পথ', 'বদল', 'ভোর', 'সমৃদ্ধি', 'বল', 'যুদ্ধ', 'প্রশ্ন', 'দরজা', 'খোদা', 'মন', 'খাদ্য', 'বসা', 'রং', 'মুখ', 'পর্ব', 'পাথর', 'সৌন্দর্য', 'সময়', 'পথিক', 'শাখা', 'মাস', 'খোঁজা', 'বাঁধা', 'সুন্দর', 'লম্বা', 'ঝগড়া', 'শান্তি', 'ভালোবাসা', 'আনন্দ', 'বন্ধু', 'শত্রু', 'শিক্ষক', 'ছাত্র', 'বাড়ি', 'বাইরে', 'বাজার', 'দোকান', 'ব্যবসা', 'চাকরি', 'সাহস', 'ভয়', 'বুদ্ধি', 'রাজ্য', 'শহর', 'গ্রাম', 'উপত্যকা', 'রাত', 'বীজ', 'পাতা', 'প্রাণী', 'কক্ষ', 'ভাই', 'বোন', 'রাজা', 'রাণী', 'মেয়ে', 'ছেলে', 'জানাান', 'আসা', 'শোনা', 'লুকানো', 'বোনা', 'হারানো', 'গলানো', 'দৌড়ানো', 'পাঠানো', 'দাঁড়ানো', 'পরিধান', 'ঝাঁকানো', 'বাজানো', 'চালানো', 'শেখা', 'কাটা', 'পাওয়া', 'খাওয়ানো', 'খাওয়া', 'টানা', 'ভর্ৎসনা', 'পোড়ানো', 'কেনা', 'পৌছানো', 'মারা', 'ভাঙা', 'আনা', 'ডাকা', 'প্রকাশ', 'বছর', 'বিরুদ্ধে', 'নৈতিক', 'ভাগ্যবান', 'বন্দী', 'ওড়া', 'লাফানো', 'হাসা', 'সক্ষমতা', 'অনুপস্থিতি', 'প্রাচুর্য', 'প্রবেশ', 'দুর্ঘটনা', 'হিসাব', 'ব্যথা', 'ভর্তি', 'সুবিধা', 'উপদেশ', 'স্নেহ', 'চুক্তি', 'ক্রোধ', 'দুশ্চিন্তা', 'অনুমান', 'প্রতিশ্রুতি', 'সচেতনতা', 'অবিবাহিত', 'মালপত্র', 'দেউলে', 'প্রতিবন্ধকতা', 'ভিত্তি', 'অস্তিত্ব', 'আশীর্বাদ', 'উপকার', 'বিশ্বাসঘাতক', 'তিক্ততা', 'অবরোধ', 'সাহসিকতা', 'বিদ্যুৎ', 'অনুষ্ঠিত', 'অংশ', 'সংবাদ', 'আটক', 'উপস্থিত', 'একটা', 'নিশ্চিত', 'ছাড়া', 'হত্যা', 'পাশাপাশি', 'রাজনৈতিক', 'নির্বাহী', 'অনলাইন', 'অবস্থায়', 'প্রার্থী', 'নারী', 'প্রতিনিধি', 'গ্রহণ', 'দক্ষিণ', 'সম্ভব', 'আদালত', 'নেয়া', 'উন্নয়ন', 'কেন্দ্রীয়', 'ঘটনার', 'প্রেসিডেন্ট', 'কমিটি', 'সৃষ্টি', 'কম', 'অর্থ', 'পরীক্ষা', 'অতিরিক্ত', 'অভিযান', 'সংখ্যা', 'পালন', 'সড়ক', 'সংসদ', 'মার্কিন', 'হামলা', 'আইন', 'বর্তমান', 'স্ত্রী', 'দুপুরে', 'বিক্রি', 'ভোট', 'বিশ্ব', 'সহকারী', 'আয়োজন', 'মেডিকেল', 'শিশু', 'সম্পর্কে', 'উল্লেখ', 'দাম', 'আবেদন', 'সূত্র', 'অবস্থান', 'সচিব', 'প্রমুখ', 'পক্ষ', 'সংশ্লিষ্ট', 'দ্রুত', 'বিপক্ষে', 'নিরাপত্তা', 'প্রতিষ্ঠান', 'ছবি', 'পদে', 'কার্যক্রম', 'যোগাযোগ', 'ছোট', 'নিজ', 'নেন', 'এলাকা', 'আহ্বান', 'বাজারে', 'আগের', 'আশা', 'অতিথি', 'এসেছে', 'দশমিক', 'সামাজিক', 'বিচার', 'সারা', 'মূল', 'কেন্দ্র', 'নির্মাণ', 'কর্মসূচি', 'যোগ', 'শিকার', 'আসা', 'বিভাগ', 'ছয়', 'জয়', 'নিয়োগ', 'তিনটি', 'লোকজন', 'গঠন', 'চলতি', 'পরিমাণ', 'পরিবার', 'বেলা', 'জড়িত', 'সংস্থা', 'উইকেট', 'করেছি', 'হামলার', 'মালিক', 'ছেড়ে', 'মন্ত্রী', 'স্কুল', 'বাহিনীর', 'অফিস', 'মহানগর', 'ব্যক্তি', 'বাড়ির', 'নির্বাচিত', 'হাত', 'নিয়েছে', 'মধ্য', 'আসছে', 'পরিষদ', 'নেয়', 'অফিসার', 'ভেঙে', 'মৃত', 'ধারণা', 'প্রকল্প', 'প্রতিটি', 'গাড়ি', 'বাস', 'জীবন', 'উদ্যোগ', 'বেসরকারি', 'কমিশন', 'সিনিয়র', 'পৌর', 'হারিয়ে', 'ব্যাপক', 'হার', 'পরিবেশ', 'অধিনায়ক', 'ভাগ', 'ফেলে', 'করায়', 'বৃদ্ধি', 'দায়ের', 'জমা', 'উচ্চ', 'প্রস্তাব', 'বেড়ে', 'জমি', 'শহীদ', 'সভা', 'গান', 'লক্ষ্য', 'কৃষি', 'মহিলা', 'অবৈধ', 'পুরো', 'উদ্বোধন', 'চলাচল', 'মন্তব্য', 'বিক্ষোভ', 'বৈঠক', 'ঘণ্টা', 'দীর্ঘ', 'খাবার', 'ফুটবল', 'সেরা', 'অব্যাহত', 'অ্যাডভোকেট', 'সরাসরি', 'প্রয়োজনীয়', 'অধিকার', 'গোলাম', 'গোল', 'সংগঠন', 'কমে', 'বিমান', 'স্বামী', 'বাহিনী', 'বেড়েছে', 'নিয়মিত', 'লেখা', 'বিরোধী', 'চূড়ান্ত', 'মুক্তি', 'ক্ষমতা', 'স্বীকার', 'পরিচয়', 'একাধিক', 'সঠিক', 'পারেনি', 'নেতারা', 'গভীর', 'দিবস', 'রক্ষা', 'জারি', 'আদেশ', 'ভিডিও', 'কার্যকর', 'প্রথমে', 'কেটে', 'ভাড়া', 'স্থায়ী', 'কিলোমিটার', 'সম্পন্ন', 'পরামর্শ', 'কঠিন', 'ভাইস', 'পথ', 'ভুল', 'অর্জন', 'শীর্ষ', 'গোয়েন্দা', 'অপর', 'অনুমোদন', 'সর্বোচ্চ', 'শতাধিক', 'প্রেস', 'সংকট', 'ক্ষতিগ্রস্ত', 'দখল', 'নির্যাতন', 'সর্বশেষ', 'স্বাধীনতা', 'এবারের', 'গ্যাস', 'প্রতিবাদ', 'প্রমাণ', 'বোর্ড', 'মূল্য', 'চাপ', 'বয়স', 'বিকাল', 'কঠোর', 'সমর্থন', 'বিয়ে', 'স্বপ্ন', 'সম্ভাবনা', 'সমাবেশ', 'চন্দ্র', 'অপরাধ', 'জরিমানা', 'জন্ম', 'দৈনিক', 'ক্লাব', 'ব্যক্তিগত', 'ওষুধ', 'নির্ধারিত', 'সেনা', 'হাজির', 'শাখার', 'বিজ্ঞান', 'ভেতরে', 'কোম্পানি', 'আয়', 'জয়ের', 'চিঠি', 'নির্ধারণ', 'পার', 'বয়সী', 'শিশুদের', 'গুরুত্ব', 'উল্লেখ্য', 'শুভ', 'চরম', 'বিনিয়োগ', 'সন্তান', 'আপিল', 'বাড়ছে', 'জনপ্রিয়', 'উন্নত', 'স্থগিত', 'পাস', 'রাস্তা', 'পদ', 'দূর', 'মিনিট', 'স্থাপন', 'ব্যর্থ', 'নিজস্ব', 'শক্তিশালী', 'ভূমি', 'আশঙ্কা', 'জোর', 'শেয়ার', 'একসঙ্গে', 'প্রবাসী', 'মত', 'মাটি', 'বারবার', 'প্রাণ', 'পরিচালিত', 'কম্পিউটার', 'দমন', 'সার্বিক', 'নিয়েই', 'সক্ষম', 'সভাপতিত্ব', 'পরের', 'ওয়ার্ড', 'সমাজ', 'রাজি', 'সংযোগ', 'পররাষ্ট্র']  # Add 500 Bengali words here
english_words = ['ignorance', 'permission', 'request', 'darkness', 'injustice', 'illiteracy', 'sky', 'fire', 'light', 'coffee', 'damage', 'drought', 'hot', 'tree', 'house', 'hatred', 'tea', 'ask', 'knowledge', 'tempest', 'storm', 'risk', 'poverty', 'sadness', 'milk', 'country', 'river', 'instruction', 'certainty', 'prohibition', 'justice', 'water', 'mountain', 'love', 'fruit', 'flower', 'flood', 'air', 'hindrance', 'faith', 'rain', 'expense', 'rice', 'earthquake', 'fish', 'human', 'meat', 'falsehood', 'bread', 'gain', 'education', 'cold', 'saving', 'truth', 'doubt', 'problem', 'solution', 'sea', 'wealth', 'help', 'happiness', 'opportunity', 'come', 'speak', 'sleep', 'walk', 'run', 'play', 'read', 'far', 'above', 'below', 'mood', 'walk', 'light', 'game', 'road', 'change', 'morning', 'prosperity', 'strength', 'war', 'question', 'door', 'dig', 'mind', 'food', 'sit', 'color', 'face', 'episode', 'stone', 'beauty', 'time', 'traveler', 'branch', 'month', 'search', 'tie', 'beautiful', 'tall', 'quarrel', 'peace', 'love', 'joy', 'friend', 'foe', 'teacher', 'student', 'home', 'outside', 'market', 'shop', 'business', 'job', 'courage', 'fear', 'intelligence', 'state', 'city', 'village', 'valley', 'night', 'seed', 'leaf', 'animal', 'room', 'brother', 'sister', 'king', 'queen', 'girl', 'boy', 'inform', 'come', 'hear', 'hide', 'knit', 'lose', 'melt', 'run', 'send', 'stand', 'wear', 'shake', 'ring', 'lead', 'learn', 'hew', 'get', 'feed', 'eat', 'draw', 'chide', 'burn', 'buy', 'arrive', 'beat', 'break', 'bring', 'call', 'expression', 'year', 'against', 'ethical', 'lucky', 'captive', 'fly', 'jump', 'laugh', 'ability', 'absence', 'abundance', 'access', 'accident', 'account', 'ache', 'admission', 'advantage', 'advice', 'affection', 'agreement', 'anger', 'anxiety', 'assume', 'assurance', 'awareness', 'bachelor', 'baggage', 'bankrupt', 'barrier', 'basement', 'being', 'benediction', 'benefit', 'betrayer', 'bitterness', 'blockade', 'boldness', 'electricity', 'held', 'part', 'news', 'detention', 'present', 'one', 'sure', 'except', 'murder', 'alongside', 'political', 'executive', 'online', 'condition', 'candidate', 'woman', 'representative', 'accept', 'south', 'possible', 'court', 'take', 'development', 'central', 'event', 'president', 'committee', 'creation', 'less', 'money', 'test', 'extra', 'campaign', 'number', 'keeping', 'road', 'parliament', 'us', 'attack', 'law', 'current', 'wife', 'noon', 'sale', 'vote', 'world', 'assistant', 'arrangement', 'medical', 'child', 'about', 'mention', 'price', 'application', 'formula', 'location', 'secretary', 'others', 'side', 'concerned', 'quickly', 'against', 'safety', 'institution', 'picture', 'position', 'activities', 'communication', 'small', 'own', 'take', 'area', 'invitation', 'market', 'previous', 'hope', 'guest', 'come', 'decimal', 'social', 'trial', 'all', 'principal', 'center', 'construction', 'schedule', 'addition', 'victim', 'coming', 'division', 'six', 'victory', 'appointment', 'three', 'people', 'formation', 'current', 'quantity', 'family', 'afternoon', 'involved', 'organization', 'wicket', 'done', 'attack', 'owner', 'left', 'minister', 'school', 'army', 'office', 'metropolitan', 'person', 'house', 'elected', 'hand', 'taken', 'middle', 'coming', 'council', 'takes', 'officer', 'broken', 'dead', 'idea', 'project', 'every', 'car', 'bus', 'life', 'initiative', 'private', 'commission', 'senior', 'municipal', 'lost', 'widespread', 'loss', 'environment', 'captain', 'share', 'throwing', 'causes', 'growth', 'filed', 'deposited', 'high', 'proposal', 'increased', 'land', 'martyr', 'meeting', 'song', 'goal', 'agriculture', 'woman', 'illegal', 'whole', 'opening', 'movement', 'comment', 'protest', 'meeting', 'hour', 'long', 'food', 'football', 'best', 'continuous', 'advocate', 'directly', 'necessary', 'right', 'slave', 'goal', 'organization', 'decreased', 'airplane', 'husband', 'forces', 'increased', 'regular', 'written', 'opponent', 'final', 'freedom', 'power', 'admitted', 'identity', 'multiple', 'correct', 'couldnt', 'leaders', 'deep', 'day', 'protection', 'issued', 'order', 'video', 'effective', 'first', 'cut', 'rent', 'permanent', 'kilometer', 'complete', 'advice', 'hard', 'vice', 'way', 'wrong', 'achievement', 'top', 'detective', 'other', 'approval', 'highest', 'hundreds', 'press', 'crisis', 'injured', 'occupation', 'torture', 'latest', 'independence', 'this', 'gas', 'protest', 'proof', 'board', 'value', 'pressure', 'age', 'afternoon', 'harsh', 'support', 'wedding', 'dream', 'possibility', 'assembly', 'moon', 'crime', 'fine', 'birth', 'daily', 'club', 'private', 'medicine', 'scheduled', 'army', 'present', 'branch', 'science', 'inside', 'company', 'income', 'victory', 'letter', 'determination', 'can', 'aged', 'children', 'importance', 'noteworthy', 'auspicious', 'extreme', 'investment', 'child', 'appeal', 'increasing', 'popular', 'developed', 'adjourned', 'pass', 'street', 'post', 'far', 'minute', 'establishment', 'failure', 'own', 'strong', 'land', 'fear', 'force', 'share', 'together', 'expat', 'opinion', 'soil', 'repeatedly', 'life', 'managed', 'computer', 'suppression', 'overall', 'with', 'able', 'presidency', 'next', 'ward', 'society', 'agreed', 'connection', 'foreign']  # Add 500 English words here


# Create empty lists to store results
positive_results = []
negative_results = []

# Loop through the word pairs
for bengali_word, english_word in zip(bengali_words, english_words):
    # Find the nearest neighbors of the English word in the English Word2Vec model
    english_neighbors = google_news.most_similar(english_word, topn=10)
    
    # Find the nearest neighbors of the Bengali word in the Bengali Word2Vec model
    bengali_neighbors = model.wv.most_similar(bengali_word, topn=2)
    
    # Iterate through Bengali neighbors
    for bengali_neighbor, _ in bengali_neighbors:
        # Get the word vectors for the Bengali word neighbor and the English word neighbors
        bengali_neighbor_vector = model.wv.get_vector(bengali_neighbor)
        english_neighbors_vectors = [google_news.get_vector(word) for word, _ in english_neighbors]
        
        # Calculate the cosine similarities between the Bengali neighbor and the English neighbors
        cosine_similarities = cosine_similarity([bengali_neighbor_vector], np.array(english_neighbors_vectors))
        
        # Iterate through English neighbors and their cosine similarities
        for eng_neighbor, similarity in zip(english_neighbors, cosine_similarities[0]):
            result = {
                'Bengali Word': bengali_word,
                'Bengali Neighbor': bengali_neighbor,
                'English Word': english_word,
                'English Neighbor': eng_neighbor[0],
                'Cosine Similarity': similarity
            }
            if similarity >= 0:
                positive_results.append(result)
            else:
                negative_results.append(result)

# Create DataFrames for positive and negative results
positive_df = pd.DataFrame(positive_results)
negative_df = pd.DataFrame(negative_results)

# Create an Excel writer
writer = pd.ExcelWriter('banglaNeiborword_cosine_similarities1.xlsx', engine='xlsxwriter')

# Save DataFrames to separate sheets in the Excel file
positive_df.to_excel(writer, sheet_name='Positive_Cosine_Similarity', index=False)
negative_df.to_excel(writer, sheet_name='Negative_Cosine_Similarity', index=False)

# Close the writer to save the changes
writer.close()

# Direct translation word cosine similarity 



In [ ]:
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# Define a list of 500 Bengali words and English words
bengali_words = ['অজ্ঞতা', 'অনুমতি', 'অনুরোধ', 'অন্ধকার', 'অন্যায়', 'অশিক্ষা', 'আকাশ', 'আগুন', 'আলো', 'কফি', 'ক্ষতি', 'খরা', 'গরম', 'গাছ', 'ঘর', 'ঘৃণা', 'চা', 'জিজ্ঞাসা', 'জ্ঞান', 'ঝঞ্ঝা', 'ঝড়', 'ঝুঁকি', 'দারিদ্র্য', 'দুঃখ', 'দুধ', 'দেশ', 'নদী', 'নির্দেশ', 'নিশ্চয়তা', 'নিষেধাজ্ঞা', 'ন্যায়', 'পানি', 'পাহাড়', 'প্রেম', 'ফল', 'ফুল', 'বন্যা', 'বাতাস', 'বাধা', 'বিশ্বাস', 'বৃষ্টি', 'ব্যয়', 'ভাত', 'ভূমিকম্প', 'মাছ', 'মানুষ', 'মাংস', 'মিথ্যা', 'রুটি', 'লাভ', 'শিক্ষা', 'শীত', 'সঞ্চয়', 'সত্য', 'সন্দেহ', 'সমস্যা', 'সমাধান', 'সমুদ্র', 'সম্পদ', 'সাহায্য', 'সুখ', 'সুযোগ', 'আসতে', 'কথা', 'ঘুমানো', 'চলতে', 'দৌড়াতে', 'খেলতে', 'পড়া', 'দূরে', 'ওপর', 'নীচে', 'মেজাজ', 'চলা', 'আলো', 'খেলা', 'পথ', 'বদল', 'ভোর', 'সমৃদ্ধি', 'বল', 'যুদ্ধ', 'প্রশ্ন', 'দরজা', 'খোদা', 'মন', 'খাদ্য', 'বসা', 'রং', 'মুখ', 'পর্ব', 'পাথর', 'সৌন্দর্য', 'সময়', 'পথিক', 'শাখা', 'মাস', 'খোঁজা', 'বাঁধা', 'সুন্দর', 'লম্বা', 'ঝগড়া', 'শান্তি', 'ভালোবাসা', 'আনন্দ', 'বন্ধু', 'শত্রু', 'শিক্ষক', 'ছাত্র', 'বাড়ি', 'বাইরে', 'বাজার', 'দোকান', 'ব্যবসা', 'চাকরি', 'সাহস', 'ভয়', 'বুদ্ধি', 'রাজ্য', 'শহর', 'গ্রাম', 'উপত্যকা', 'রাত', 'বীজ', 'পাতা', 'প্রাণী', 'কক্ষ', 'ভাই', 'বোন', 'রাজা', 'রাণী', 'মেয়ে', 'ছেলে', 'জানাান', 'আসা', 'শোনা', 'লুকানো', 'বোনা', 'হারানো', 'গলানো', 'দৌড়ানো', 'পাঠানো', 'দাঁড়ানো', 'পরিধান', 'ঝাঁকানো', 'বাজানো', 'চালানো', 'শেখা', 'কাটা', 'পাওয়া', 'খাওয়ানো', 'খাওয়া', 'টানা', 'ভর্ৎসনা', 'পোড়ানো', 'কেনা', 'পৌছানো', 'মারা', 'ভাঙা', 'আনা', 'ডাকা', 'প্রকাশ', 'বছর', 'বিরুদ্ধে', 'নৈতিক', 'ভাগ্যবান', 'বন্দী', 'ওড়া', 'লাফানো', 'হাসা', 'সক্ষমতা', 'অনুপস্থিতি', 'প্রাচুর্য', 'প্রবেশ', 'দুর্ঘটনা', 'হিসাব', 'ব্যথা', 'ভর্তি', 'সুবিধা', 'উপদেশ', 'স্নেহ', 'চুক্তি', 'ক্রোধ', 'দুশ্চিন্তা', 'অনুমান', 'প্রতিশ্রুতি', 'সচেতনতা', 'অবিবাহিত', 'মালপত্র', 'দেউলে', 'প্রতিবন্ধকতা', 'ভিত্তি', 'অস্তিত্ব', 'আশীর্বাদ', 'উপকার', 'বিশ্বাসঘাতক', 'তিক্ততা', 'অবরোধ', 'সাহসিকতা', 'বিদ্যুৎ', 'অনুষ্ঠিত', 'অংশ', 'সংবাদ', 'আটক', 'উপস্থিত', 'একটা', 'নিশ্চিত', 'ছাড়া', 'হত্যা', 'পাশাপাশি', 'রাজনৈতিক', 'নির্বাহী', 'অনলাইন', 'অবস্থায়', 'প্রার্থী', 'নারী', 'প্রতিনিধি', 'গ্রহণ', 'দক্ষিণ', 'সম্ভব', 'আদালত', 'নেয়া', 'উন্নয়ন', 'কেন্দ্রীয়', 'ঘটনার', 'প্রেসিডেন্ট', 'কমিটি', 'সৃষ্টি', 'কম', 'অর্থ', 'পরীক্ষা', 'অতিরিক্ত', 'অভিযান', 'সংখ্যা', 'পালন', 'সড়ক', 'সংসদ', 'মার্কিন', 'হামলা', 'আইন', 'বর্তমান', 'স্ত্রী', 'দুপুরে', 'বিক্রি', 'ভোট', 'বিশ্ব', 'সহকারী', 'আয়োজন', 'মেডিকেল', 'শিশু', 'সম্পর্কে', 'উল্লেখ', 'দাম', 'আবেদন', 'সূত্র', 'অবস্থান', 'সচিব', 'প্রমুখ', 'পক্ষ', 'সংশ্লিষ্ট', 'দ্রুত', 'বিপক্ষে', 'নিরাপত্তা', 'প্রতিষ্ঠান', 'ছবি', 'পদে', 'কার্যক্রম', 'যোগাযোগ', 'ছোট', 'নিজ', 'নেন', 'এলাকা', 'আহ্বান', 'বাজারে', 'আগের', 'আশা', 'অতিথি', 'এসেছে', 'দশমিক', 'সামাজিক', 'বিচার', 'সারা', 'মূল', 'কেন্দ্র', 'নির্মাণ', 'কর্মসূচি', 'যোগ', 'শিকার', 'আসা', 'বিভাগ', 'ছয়', 'জয়', 'নিয়োগ', 'তিনটি', 'লোকজন', 'গঠন', 'চলতি', 'পরিমাণ', 'পরিবার', 'বেলা', 'জড়িত', 'সংস্থা', 'উইকেট', 'করেছি', 'হামলার', 'মালিক', 'ছেড়ে', 'মন্ত্রী', 'স্কুল', 'বাহিনীর', 'অফিস', 'মহানগর', 'ব্যক্তি', 'বাড়ির', 'নির্বাচিত', 'হাত', 'নিয়েছে', 'মধ্য', 'আসছে', 'পরিষদ', 'নেয়', 'অফিসার', 'ভেঙে', 'মৃত', 'ধারণা', 'প্রকল্প', 'প্রতিটি', 'গাড়ি', 'বাস', 'জীবন', 'উদ্যোগ', 'বেসরকারি', 'কমিশন', 'সিনিয়র', 'পৌর', 'হারিয়ে', 'ব্যাপক', 'হার', 'পরিবেশ', 'অধিনায়ক', 'ভাগ', 'ফেলে', 'করায়', 'বৃদ্ধি', 'দায়ের', 'জমা', 'উচ্চ', 'প্রস্তাব', 'বেড়ে', 'জমি', 'শহীদ', 'সভা', 'গান', 'লক্ষ্য', 'কৃষি', 'মহিলা', 'অবৈধ', 'পুরো', 'উদ্বোধন', 'চলাচল', 'মন্তব্য', 'বিক্ষোভ', 'বৈঠক', 'ঘণ্টা', 'দীর্ঘ', 'খাবার', 'ফুটবল', 'সেরা', 'অব্যাহত', 'অ্যাডভোকেট', 'সরাসরি', 'প্রয়োজনীয়', 'অধিকার', 'গোলাম', 'গোল', 'সংগঠন', 'কমে', 'বিমান', 'স্বামী', 'বাহিনী', 'বেড়েছে', 'নিয়মিত', 'লেখা', 'বিরোধী', 'চূড়ান্ত', 'মুক্তি', 'ক্ষমতা', 'স্বীকার', 'পরিচয়', 'একাধিক', 'সঠিক', 'পারেনি', 'নেতারা', 'গভীর', 'দিবস', 'রক্ষা', 'জারি', 'আদেশ', 'ভিডিও', 'কার্যকর', 'প্রথমে', 'কেটে', 'ভাড়া', 'স্থায়ী', 'কিলোমিটার', 'সম্পন্ন', 'পরামর্শ', 'কঠিন', 'ভাইস', 'পথ', 'ভুল', 'অর্জন', 'শীর্ষ', 'গোয়েন্দা', 'অপর', 'অনুমোদন', 'সর্বোচ্চ', 'শতাধিক', 'প্রেস', 'সংকট', 'ক্ষতিগ্রস্ত', 'দখল', 'নির্যাতন', 'সর্বশেষ', 'স্বাধীনতা', 'এবারের', 'গ্যাস', 'প্রতিবাদ', 'প্রমাণ', 'বোর্ড', 'মূল্য', 'চাপ', 'বয়স', 'বিকাল', 'কঠোর', 'সমর্থন', 'বিয়ে', 'স্বপ্ন', 'সম্ভাবনা', 'সমাবেশ', 'চন্দ্র', 'অপরাধ', 'জরিমানা', 'জন্ম', 'দৈনিক', 'ক্লাব', 'ব্যক্তিগত', 'ওষুধ', 'নির্ধারিত', 'সেনা', 'হাজির', 'শাখার', 'বিজ্ঞান', 'ভেতরে', 'কোম্পানি', 'আয়', 'জয়ের', 'চিঠি', 'নির্ধারণ', 'পার', 'বয়সী', 'শিশুদের', 'গুরুত্ব', 'উল্লেখ্য', 'শুভ', 'চরম', 'বিনিয়োগ', 'সন্তান', 'আপিল', 'বাড়ছে', 'জনপ্রিয়', 'উন্নত', 'স্থগিত', 'পাস', 'রাস্তা', 'পদ', 'দূর', 'মিনিট', 'স্থাপন', 'ব্যর্থ', 'নিজস্ব', 'শক্তিশালী', 'ভূমি', 'আশঙ্কা', 'জোর', 'শেয়ার', 'একসঙ্গে', 'প্রবাসী', 'মত', 'মাটি', 'বারবার', 'প্রাণ', 'পরিচালিত', 'কম্পিউটার', 'দমন', 'সার্বিক', 'নিয়েই', 'সক্ষম', 'সভাপতিত্ব', 'পরের', 'ওয়ার্ড', 'সমাজ', 'রাজি', 'সংযোগ', 'পররাষ্ট্র']  # Add 500 Bengali words here
english_words = ['ignorance', 'permission', 'request', 'darkness', 'injustice', 'illiteracy', 'sky', 'fire', 'light', 'coffee', 'damage', 'drought', 'hot', 'tree', 'house', 'hatred', 'tea', 'ask', 'knowledge', 'tempest', 'storm', 'risk', 'poverty', 'sadness', 'milk', 'country', 'river', 'instruction', 'certainty', 'prohibition', 'justice', 'water', 'mountain', 'love', 'fruit', 'flower', 'flood', 'air', 'hindrance', 'faith', 'rain', 'expense', 'rice', 'earthquake', 'fish', 'human', 'meat', 'falsehood', 'bread', 'gain', 'education', 'cold', 'saving', 'truth', 'doubt', 'problem', 'solution', 'sea', 'wealth', 'help', 'happiness', 'opportunity', 'come', 'speak', 'sleep', 'walk', 'run', 'play', 'read', 'far', 'above', 'below', 'mood', 'walk', 'light', 'game', 'road', 'change', 'morning', 'prosperity', 'strength', 'war', 'question', 'door', 'dig', 'mind', 'food', 'sit', 'color', 'face', 'episode', 'stone', 'beauty', 'time', 'traveler', 'branch', 'month', 'search', 'tie', 'beautiful', 'tall', 'quarrel', 'peace', 'love', 'joy', 'friend', 'foe', 'teacher', 'student', 'home', 'outside', 'market', 'shop', 'business', 'job', 'courage', 'fear', 'intelligence', 'state', 'city', 'village', 'valley', 'night', 'seed', 'leaf', 'animal', 'room', 'brother', 'sister', 'king', 'queen', 'girl', 'boy', 'inform', 'come', 'hear', 'hide', 'knit', 'lose', 'melt', 'run', 'send', 'stand', 'wear', 'shake', 'ring', 'lead', 'learn', 'hew', 'get', 'feed', 'eat', 'draw', 'chide', 'burn', 'buy', 'arrive', 'beat', 'break', 'bring', 'call', 'expression', 'year', 'against', 'ethical', 'lucky', 'captive', 'fly', 'jump', 'laugh', 'ability', 'absence', 'abundance', 'access', 'accident', 'account', 'ache', 'admission', 'advantage', 'advice', 'affection', 'agreement', 'anger', 'anxiety', 'assume', 'assurance', 'awareness', 'bachelor', 'baggage', 'bankrupt', 'barrier', 'basement', 'being', 'benediction', 'benefit', 'betrayer', 'bitterness', 'blockade', 'boldness', 'electricity', 'held', 'part', 'news', 'detention', 'present', 'one', 'sure', 'except', 'murder', 'alongside', 'political', 'executive', 'online', 'condition', 'candidate', 'woman', 'representative', 'accept', 'south', 'possible', 'court', 'take', 'development', 'central', 'event', 'president', 'committee', 'creation', 'less', 'money', 'test', 'extra', 'campaign', 'number', 'keeping', 'road', 'parliament', 'us', 'attack', 'law', 'current', 'wife', 'noon', 'sale', 'vote', 'world', 'assistant', 'arrangement', 'medical', 'child', 'about', 'mention', 'price', 'application', 'formula', 'location', 'secretary', 'others', 'side', 'concerned', 'quickly', 'against', 'safety', 'institution', 'picture', 'position', 'activities', 'communication', 'small', 'own', 'take', 'area', 'invitation', 'market', 'previous', 'hope', 'guest', 'come', 'decimal', 'social', 'trial', 'all', 'principal', 'center', 'construction', 'schedule', 'addition', 'victim', 'coming', 'division', 'six', 'victory', 'appointment', 'three', 'people', 'formation', 'current', 'quantity', 'family', 'afternoon', 'involved', 'organization', 'wicket', 'done', 'attack', 'owner', 'left', 'minister', 'school', 'army', 'office', 'metropolitan', 'person', 'house', 'elected', 'hand', 'taken', 'middle', 'coming', 'council', 'takes', 'officer', 'broken', 'dead', 'idea', 'project', 'every', 'car', 'bus', 'life', 'initiative', 'private', 'commission', 'senior', 'municipal', 'lost', 'widespread', 'loss', 'environment', 'captain', 'share', 'throwing', 'causes', 'growth', 'filed', 'deposited', 'high', 'proposal', 'increased', 'land', 'martyr', 'meeting', 'song', 'goal', 'agriculture', 'woman', 'illegal', 'whole', 'opening', 'movement', 'comment', 'protest', 'meeting', 'hour', 'long', 'food', 'football', 'best', 'continuous', 'advocate', 'directly', 'necessary', 'right', 'slave', 'goal', 'organization', 'decreased', 'airplane', 'husband', 'forces', 'increased', 'regular', 'written', 'opponent', 'final', 'freedom', 'power', 'admitted', 'identity', 'multiple', 'correct', 'could', "n't", 'leaders', 'deep', 'day', 'protection', 'issued', 'order', 'video', 'effective', 'first', 'cut', 'rent', 'permanent', 'kilometer', 'complete', 'advice', 'hard', 'vice', 'way', 'wrong', 'achievement', 'top', 'detective', 'other', 'approval', 'highest', 'hundreds', 'press', 'crisis', 'injured', 'occupation', 'torture', 'latest', 'independence', 'this', 'gas', 'protest', 'proof', 'board', 'value', 'pressure', 'age', 'afternoon', 'harsh', 'support', 'wedding', 'dream', 'possibility', 'assembly', 'moon', 'crime', 'fine', 'birth', 'daily', 'club', 'private', 'medicine', 'scheduled', 'army', 'present', 'branch', 'science', 'inside', 'company', 'income', 'victory', 'letter', 'determination', 'can', 'aged', 'children', 'importance', 'noteworthy', 'auspicious', 'extreme', 'investment', 'child', 'appeal', 'increasing', 'popular', 'developed', 'adjourned', 'pass', 'street', 'post', 'far', 'minute', 'establishment', 'failure', 'own', 'strong', 'land', 'fear', 'force', 'share', 'together', 'expat', 'opinion', 'soil', 'repeatedly', 'life', 'managed', 'computer', 'suppression', 'overall', 'with', 'able', 'presidency', 'next', 'ward', 'society', 'agreed', 'connection', 'foreign']  # Add 500 English words here

# Calculate the cosine similarity for each pair of Bengali and English words
print("Cosine Similarities:")
for bengali_word, english_word in zip(bengali_words, english_words):
    # Find the top 10 nearest neighbors of the English word in the English Word2Vec model
    english_neighbors = google_news.most_similar(english_word, topn=10)

    # Get the word vectors for the Bengali word and the English word's neighbors
    bengali_vector = model.wv.get_vector(bengali_word)
    english_neighbors_vectors = [google_news.get_vector(word) for word, _ in english_neighbors]

    # Calculate the cosine similarities between the Bengali word and the English word's neighbors
    cosine_similarities = cosine_similarity(np.array([bengali_vector]), np.array(english_neighbors_vectors))

    # Print the results
    print(f"For Bengali word '{bengali_word}' and English word '{english_word}':")
    for neighbor, similarity in zip(english_neighbors, cosine_similarities[0]):
        print(f"  {neighbor[0]}: {similarity:.4f}")


## Finding the most similar english words for a Bangla word negating the garbage data from GoogleNews

In [ ]:
import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity



with open("words_alpha.txt", "r", encoding="utf-8") as file:
    valid_english_words = set(file.read().split())

def find_top_similar_words(bangla_word, google_news, valid_english_words, top_n=50):
    try:
        # Get the embedding for the Bangla word
        bangla_embedding = model.wv[bangla_word]

        # Filter valid English words that exist in the Google News Word2Vec model
        valid_english_vocab = [word for word in valid_english_words if word in google_news]

        if not valid_english_vocab:
            print("No valid English words found in the model.")
            return []

        # Calculate cosine similarity between the Bangla word and valid English words
        similarities = cosine_similarity([bangla_embedding], [google_news[word] for word in valid_english_vocab])

        # Get the indices of the top N similar words
        similar_word_indices = np.argsort(similarities[0])[::-1][:top_n]

        # Get the actual words corresponding to the indices and their similarity scores
        similar_words = [(valid_english_vocab[i], similarities[0][i]) for i in similar_word_indices]

        return similar_words
    except KeyError:
        return []

# Example usage
bangla_word = "জীবন"  # Replace with your Bangla word
similar_words = find_top_similar_words(bangla_word, google_news, valid_english_words)

for idx, (word, score) in enumerate(similar_words, 1):
    print(f"{idx}. {word}: Cosine Similarity Score = {score:.4f}")

## WordNet Path similarity check for English counterpart words for a given Bangla Word

In [ ]:
import gensim
import nltk
from nltk.corpus import wordnet
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize NLTK and download WordNet data
nltk.download('wordnet')


# Load a list of valid English words (you can create or obtain this list)
with open("words_alpha.txt", "r", encoding="utf-8") as file:
    valid_english_words = set(file.read().split())

def find_top_similar_words(bangla_word, google_news, valid_english_words, top_n=50):
    try:
        # Get the embedding for the Bangla word
        bangla_embedding = model.wv[bangla_word]

        # Filter valid English words that exist in the Google News Word2Vec model
        valid_english_vocab = [word for word in valid_english_words if word in google_news]

        if not valid_english_vocab:
            print("No valid English words found in the model.")
            return []

        # Calculate cosine similarity between the Bangla word and valid English words
        similarities = cosine_similarity([bangla_embedding], [google_news[word] for word in valid_english_vocab])

        # Get the indices of the top N similar words
        similar_word_indices = np.argsort(similarities[0])[::-1][:top_n]

        # Get the actual words corresponding to the indices
        similar_words = [valid_english_vocab[i] for i in similar_word_indices]

        return similar_words
    except KeyError:
        return []

def compare_wordnet_similarity(english_word, similar_words):
    synsets_english = wordnet.synsets(english_word)
    if synsets_english:
        wordnet_similarities = []
        for similar_word in similar_words:
            synsets_similar_word = wordnet.synsets(similar_word)
            if synsets_similar_word:
                shortest_path = None
                for synset_english in synsets_english:
                    for synset_similar_word in synsets_similar_word:
                        path_similarity = synset_english.path_similarity(synset_similar_word)
                        if path_similarity is not None:
                            if shortest_path is None or path_similarity > shortest_path:
                                shortest_path = path_similarity
                wordnet_similarities.append((similar_word, shortest_path))
            else:
                wordnet_similarities.append((similar_word, None))
        return wordnet_similarities
    else:
        print(f"No WordNet synsets found for '{english_word}'")
        return [(similar_word, None) for similar_word in similar_words]

# Example usage
bangla_word = "বাংলা"  # Replace with your Bangla word
english_word = "english"   # Replace with your English word

# Find the top 50 most similar English words for the Bangla word
top_similar_words = find_top_similar_words(bangla_word, google_news, valid_english_words, top_n=50)

if not top_similar_words:
    print(f"No valid English words found for '{bangla_word}'")
else:
    # Select the top 3 most similar English words
    top_3_similar_words = top_similar_words[:3]

    # Compare WordNet path similarity between the selected English words and the new English word
    wordnet_similarities = compare_wordnet_similarity(english_word, top_3_similar_words)

    for idx, (word, similarity) in enumerate(wordnet_similarities, 1):
        if similarity is not None:
            print(f"WordNet path similarity between '{english_word}' and '{word}': {similarity:.4f}")
        else:
            print(f"No WordNet synsets found for '{word}'")

## WordNet Shortest Distance for a English Counterpart of a given Bangla Word

In [ ]:
import gensim
import nltk
from nltk.corpus import wordnet
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity





# Load a list of valid English words (you can create or obtain this list)
with open("words_alpha.txt", "r", encoding="utf-8") as file:
    valid_english_words = set(file.read().split())

def find_top_similar_words(bangla_word, google_news, valid_english_words, top_n=50):
    try:
        # Get the embedding for the Bangla word
        bangla_embedding = model.wv[bangla_word]

        # Filter valid English words that exist in the Google News Word2Vec model
        valid_english_vocab = [word for word in valid_english_words if word in google_news]

        if not valid_english_vocab:
            print("No valid English words found in the model.")
            return []

        # Calculate cosine similarity between the Bangla word and valid English words
        similarities = cosine_similarity([bangla_embedding], [google_news[word] for word in valid_english_vocab])

        # Get the indices of the top N similar words
        similar_word_indices = np.argsort(similarities[0])[::-1][:top_n]

        # Get the actual words corresponding to the indices
        similar_words = [valid_english_vocab[i] for i in similar_word_indices]

        return similar_words
    except KeyError:
        return []

def wordnet_shortest_distance(english_word1, english_word2):
    synset1 = wordnet.synsets(english_word1)
    synset2 = wordnet.synsets(english_word2)
    if synset1 and synset2:
        shortest_distance = float('inf')
        for s1 in synset1:
            for s2 in synset2:
                distance = s1.shortest_path_distance(s2)
                if distance is not None and distance < shortest_distance:
                    shortest_distance = distance
        return shortest_distance
    else:
        return None

# Example usage
bangla_word = "সূত্র"  # Replace with your Bangla word
english_word = "tip"   # Replace with your English word

# Find the top 50 most similar English words for the Bangla word
top_similar_words = find_top_similar_words(bangla_word, google_news, valid_english_words, top_n=50)

if not top_similar_words:
    print(f"No valid English words found for '{bangla_word}'")
else:
    # Select the top 3 most similar English words
    top_3_similar_words = top_similar_words[:3]

    # Calculate the WordNet shortest distance between the selected English words and the new English word
    shortest_distances = [wordnet_shortest_distance(english_word, word) for word in top_3_similar_words]

    for idx, (word, distance) in enumerate(zip(top_3_similar_words, shortest_distances), 1):
        if distance is not None:
            print(f"WordNet shortest distance between '{english_word}' and '{word}': {distance}")
        else:
            print(f"No WordNet synsets found for '{word}'")